In [ ]:
import pandas as pd
import itertools
import string
import re
import json
import numpy as np

In [ ]:
def parse_description(description):
    opening_dates = []
    closing_dates = []
    alternate_names = []
    companies = []
    disambiguator = []
    location = []
    
    # -----------------------------------
    # Capture company:
    re_company = r"\[ *([A-Za-z](.*?[A-Za-z]))? *\]"
    
    descr_beginning = description[:20] # Company is always at the beginning
    if re.search(re_company, descr_beginning):
        companies.append(re.search(re_company, descr_beginning).group(1))
    
    # -----------------------------------
    # Remove non-printable characters:
    description = ''.join([x if x in string.printable else ' ' for x in description])
    
    # -----------------------------------
    # Remove notes (in parentheses), and text in curly and square brackets:
    description = re.sub(r'\([^)]*?\)', '', description)
    description = re.sub(r'\[[^)]*?\]', '', description)
    description = re.sub(r'\{[^)]*?\}', '', description)
    
    # -----------------------------------
    # Remove extra white spaces:
    description = re.sub(' +', ' ', description)
    
    # -----------------------------------
    # Capture opening dates:
    re_op_date_standard = r"\b(?:[Rr]e)?[Oo]p(?:en(?:ed)?)?\b(?: \[(?:[A-Z].*?)\])? *((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    re_op_date_reverse = r"\b(?:[Rr]e)?[Oo]p(?:en(?:ed)?)?\b(?: \[([?:A-Z].*?)\])? *(?:(?:[12][0-9]{3}) *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[0-9]{1,2})?)"
    re_op_date_nomark = r"\[[A-Za-z\&\;]+\] *((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    re_op_date_flexible = r"\b(?:[Rr]e)?[Oo]p(?:en(?:ed)?)?\b(?: \[(?:[A-Z].*?)\])? *(?:(?:[a-z]+) +)+((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    re_op_date_flexreverse = r"\b(?:[Rr]e)?[Oo]p(?:en(?:ed)?)?\b(?: \[(?:[A-Z].*?)\])? *(?:(?:[a-z]+) +)+((?:(?:[12][0-9]{3}) *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?)? *(?:[0-9]{1,2})?))"
    re_op_date_flexnomark = r"\[[A-Za-z\&\;]+\] *(?:(?:[A-Za-z]+) +)+((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    re_op_firstinbrad = r"[Ff]irst *(?:(?:[a-z]+) +)+[Bb]rad*(?:(?:[a-z]+) +)+((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    
    opst = re.findall(re_op_date_standard, description)
    oprv = re.findall(re_op_date_reverse, description)
    opnm = re.findall(re_op_date_nomark, description)
    opfl = re.findall(re_op_date_flexible, description)
    opflrv = re.findall(re_op_date_flexreverse, description)
    opnmfl = re.findall(re_op_date_flexnomark, description)
    opfib = re.findall(re_op_firstinbrad, description)
    
    capturedOp = list(set(opst+oprv+opnm+opfl+opflrv+opnmfl))
    
    # If no openingdate has been found, add first-in-brad date if exists:
    if not capturedOp:
        capturedOp += opfib
    
    opening_dates = capturedOp
    
    # -----------------------------------
    # Capture closing dates:
    re_cl_date_standard = r"\b(?:re)?[Cc]?lo(?:sed)?\b *((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    re_cl_date_reverse = r"\b(?:re)?[Cc]?lo(?:sed)?\b *(?:(?:[12][0-9]{3}) *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[0-9]{1,2})?)"
    re_cl_date_flexible = r"\b(?:re)?[Cc]?lo(?:sed)?\b *(?:(?:[A-Za-z]+) )+((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    re_cl_date_flexreverse = r"\b(?:re)?[Cc]?lo(?:sed)?\b *(?:(?:[A-Za-z]+) )+((?:(?:[12][0-9]{3}) *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?)? *(?:[0-9]{1,2})?))"
    re_cl_date_last = r"[Ll]ast *(?:(?:[A-Za-z]+) +)*((?:[0-9]{1,2})? *(?:Jan ?(?:uary)?|Feb ?(?:ruary)?|Mar ?(?:ch)?|Apr ?(?:il)?|May ?|Jun ?(?:e)?|Jul ?(?:y)?|Aug ?(?:ust)?|Sep ?(?:tember)?|Oct ?(?:ober)?|Nov ?(?:ember)?|Dec ?(?:ember)?) *(?:[12][0-9]{3}))"
    clst = re.findall(re_cl_date_standard, description)
    clrv = re.findall(re_cl_date_reverse, description)
    clfl = re.findall(re_cl_date_flexible, description)
    clflrv = re.findall(re_cl_date_flexreverse, description)
    cllast = re.findall(re_cl_date_last, description)
    
    capturedClo = list(set(clst+clrv+clfl+clflrv))
    # If "still open" in description, add as closing date:
    if "still open" in description.lower():
        capturedClo.append("still open")
        
    # If no closing date has been found, add last-in-brad date if exists:
    if not capturedClo:
        capturedClo += cllast
        
    closing_dates = capturedClo
    
    return opening_dates, closing_dates, companies

In [ ]:
df = pd.read_pickle("quicks_processed.pkl")

openings = []
closings = []
companies = []
for i, row in df.iterrows():
    t = parse_description(row["Description"])
    openings.append(t[0])
    closings.append(t[1])
    companies.append(t[2])
    
# Add to dataframe
df_stninfo = df.copy()
df_stninfo["Opened"] = openings
df_stninfo["Closed"] = closings
df_stninfo["Company"] = companies

# Store dataframe
df_stninfo.to_pickle("quicks_stninfo.pkl")